In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from track import get_infected_img
from track import get_tracks, plot_tracks, get_events_df
from tqdm import tqdm
import sys
sys.path.insert(0, '..') # in order to be able to import from scripts.py
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [ ]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

# Tracking simulations

In [ ]:
sim_num = 500
folder = "results/data" # get from https://drive.google.com/file/d/1zznedtXpV-gMKp_eC2hkqxM-rrvjJH9w/view?usp=sharing

In [ ]:
from laptrack import LapTrack
def get_laptrack(max_distance = 32):
    lt = LapTrack(
        track_dist_metric="sqeuclidean",  # The similarity metric for particles. See `scipy.spatial.distance.cdist` for allowed values.
        splitting_dist_metric="sqeuclidean",
        merging_dist_metric="sqeuclidean",
        track_cost_cutoff=max_distance**2,
        splitting_cost_cutoff=max_distance**2,  # or False for non-splitting case
        merging_cost_cutoff= False,  # or False for non-merging case
        )
    return lt

In [ ]:
events = []
too_many_splits = []
lt = get_laptrack(max_distance = 20)

for i in tqdm(range(sim_num)):
    df = pd.read_csv(f"{folder}/out-{i}.csv")
    img = get_infected_img(df)
    track_df, split_df, merge_df = get_tracks(img, lt, roll=20, std_factor=1/6)
    if (track_df['tree_id'] > 1).any():
        too_many_splits.append(i)
    if len(split_df) == 0:
        events.append(pd.DataFrame())
        continue
    events_df = get_events_df(track_df, split_df)
    events.append(events_df)

In [ ]:
df_results = pd.concat(events, names=['sim_id'], keys=range(sim_num))
df_results.to_csv("results/results.csv")
np.savetxt("sim_id_errors.txt",np.array(too_many_splits), fmt='%s')

In [ ]:
#too_many_splits = np.array([ 25,  39,  42,  52, 131, 134, 151, 163, 185, 203, 219, 222, 229,
#       232, 236, 237, 250, 256, 259, 263, 280, 301, 307, 335, 348, 358,
#       370, 391, 399, 425, 437, 449, 482, 490, 499])

## Testing

In [ ]:
def debug(sim_path, max_distance, roll = 20, std_factor = 1/6):
    df = pd.read_csv(sim_path)
    img = get_infected_img(df)
    get_laptrack(max_distance=max_distance)
    track_df, split_df, merge_df = get_tracks(img, lt, roll=roll, std_factor=std_factor)
    plot_tracks(track_df, split_df, merge_df)
    return track_df, split_df

In [ ]:
for i, sim_id in enumerate(too_many_splits):
    sim_path = f"results/data/out-{sim_id}.csv"
    plt.title(f"Simulation id: {sim_id}")
    debug(sim_path, 25, roll = 20, std_factor=1/6)

In [ ]:
zero_splits = df_results[df_results['tree_id'] == 0].groupby('sim_id').size().index.values
zero_splits # 309 is error rest is ok

In [ ]:
for i, sim_id in enumerate(zero_splits):
    sim_path = f"results/data/out-{sim_id}.csv"
    plt.title(f"Simulation id: {sim_id}")
    debug(sim_path, 25, roll=20)


# Get data

In [ ]:
sim_num = 500
folder = "results/data"

In [ ]:
for channel_height in [7]:

    client = VisAVisClient(
        visavis_bin=f'../target/bins/vis-a-vis-{channel_height}',
    )

    for interval in [100]:

        protocol_file_path = make_protocol(
            pulse_intervals = [interval, 1500],
            duration=4,
            out_folder='./'
        )
        
        for sim in tqdm(range(sim_num)):
            result = client.run(
                parameters_json=PARAMETERS_DEFAULT,
                protocol_file_path= protocol_file_path,
            )
            result.states.to_csv(f"{folder}/out-{sim}.csv", index=False)